Cel: Uruchomienie pierwszego (już wytrenowanego) modelu do sterowania samochodu

In [1]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from io import BytesIO
from PIL import Image
import numpy as np

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
model = load_model('model.h5')
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 58, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 76, 32)        18464     
_________________________________________________________________
activation_2 (Activation)    (None, 56, 76, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 36, 16)        4624      
____________________

In [3]:
sio = socketio.Server()
app = Flask(__name__)

def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle), #wysyłanie informacji o kącie skrętu kierownicy do symulatora
                            'throttle': str(throttle) }, skip_sid=True) #wysyłanie informacji o pedale gazu do symulatora

In [7]:
def process_image(img):
    return img[10:130:2,::4,] #ograniczenie zwracanego obrazu



@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]

        decoded = base64.b64decode(image_str)
        #Konwersja obrazu z formatu string na obraz
        image = Image.open(BytesIO(decoded))
        
        #Konwersja obrazu na macierz
        image_array = np.asarray(image)
        
        print(image_array.shape)
        
        #model input: 60x80x3
        
        #plt.imshow(image_array)
        #plt.show();
        
        img = process_image(image_array)
        img_batch = np.expand_dims(img, axis=0)
        steering_angle = float(model.predict(img_batch))
        
        
        #print(data.keys())
        #steering_angle = 1.0
        throttle = 0.1
        
        if speed < 10:
            throttle = 0.6
        if speed > 15:
            throttle = -0.1

        print(throttle,steering_angle )
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)


app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(12160) wsgi starting up on http://0.0.0.0:4567
(12160) accepted ('127.0.0.1', 60715)


(160, 320, 3)
0.6 -0.08293396234512329
(160, 320, 3)
0.6 -0.08293396234512329
(160, 320, 3)
0.6 -0.08358157426118851
(160, 320, 3)
0.6 -0.08707453310489655
(160, 320, 3)
0.6 -0.08707453310489655
(160, 320, 3)
0.6 -0.09817614406347275
(160, 320, 3)
0.6 -0.08363943547010422
(160, 320, 3)
0.6 -0.12497302144765854
(160, 320, 3)
0.6 -0.12232588231563568
(160, 320, 3)
0.6 -0.11174041777849197
(160, 320, 3)
0.6 -0.1340801864862442
(160, 320, 3)
0.6 -0.14972035586833954
(160, 320, 3)
0.6 -0.13826671242713928
(160, 320, 3)
0.6 -0.15812402963638306
(160, 320, 3)
0.6 -0.15806418657302856
(160, 320, 3)
0.6 -0.13352705538272858
(160, 320, 3)
0.6 -0.13169296085834503
(160, 320, 3)
0.6 -0.13987304270267487
(160, 320, 3)
0.6 -0.11775297671556473
(160, 320, 3)
0.6 -0.042537424713373184
(160, 320, 3)
0.6 -0.0629921555519104
(160, 320, 3)
0.6 -0.06712371855974197
(160, 320, 3)
0.6 -0.07222172617912292
(160, 320, 3)
0.6 -0.07518745213747025
(160, 320, 3)
0.6 -0.04577674716711044
(160, 320, 3)
0.6 -0.07456

127.0.0.1 - - [22/Oct/2019 21:47:06] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 116.159878
wsgi exiting
(12160) wsgi exited, is_accepting=True


![](https://ibb.co/xCLdzzW)

![](https://cdn1.imggmi.com/uploads/2019/10/22/d7f3056b4cbdacbe69b826469c81a4a1-full.jpg)